In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
from langchain import hub
from langchain_core.messages import  HumanMessage
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import TextLoader

In [54]:
loader = TextLoader("./data.txt")
docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs) #if we need to split doc.
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [55]:
contextualize_q_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [56]:
qa_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. \
    {context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever
    )
    | qa_prompt
    | llm
)

In [58]:
chat_history = []

for i in range(10):
  
  question = input("Himalay: ")
  print("Himlay: ", question)
  ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
  print("AI Response: ",ai_msg)
  print("--------------------------------------------------------------")
  chat_history.extend([HumanMessage(content=question), ai_msg])

Himlay:  hELLO
AI Response:  content='Hi, this is South Bay Dental Office, how may I help you?'
--------------------------------------------------------------
Himlay:  i want to book apoitment
AI Response:  content="Sure, I'd be happy to assist you with that. Are you a new patient or an existing patient?"
--------------------------------------------------------------
Himlay:  existing
AI Response:  content='Great! Could you please provide me with your phone number?'
--------------------------------------------------------------
Himlay:  9624788668
AI Response:  content="Thank you for providing your phone number. Now, let's move on to scheduling your appointment. May I know your availability?"
--------------------------------------------------------------
Himlay:  saturday 9pm
AI Response:  content='I apologize, but our office hours on Saturdays are from 9:00 AM to 1:00 PM. Is there any other day or time that works for you?'
--------------------------------------------------------------

KeyboardInterrupt: 